## EXPERIMENTO 1

* Métrica Perplejidad: https://en.wikipedia.org/wiki/Perplexity#cite_ref-1

* Loss: Categorical Crossentropy (buscar más referencias para ambos)

* Arquitectura: Capa average, capa max y capa densa

### Corpus

@InProceedings{Danescu-Niculescu-Mizil+Lee:11a,

  author={Cristian Danescu-Niculescu-Mizil and Lillian Lee},

  title={Chameleons in imagined conversations:

  A new approach to understanding coordination of linguistic style in dialogs.},

  booktitle={Proceedings of the

        Workshop on Cognitive Modeling and Computational Linguistics, ACL 2011},

  year={2011}

}

In [1]:
data = """Con diez cañones por banda,
viento en popa a toda vela,
no corta el mar, sino vuela,
un velero bergantín;
bajel pirata que llaman
por su bravura el Temido
en todo el mar conocido
del uno al otro confín.
La luna en el mar riela,
en la lona gime el viento
y alza en blando movimiento
olas de plata y azul;
y ve el capitán pirata,
cantando alegre en la popa,
Asia a un lado, al otro Europa,
Y allá a su frente Estambul:
-Navega, velero mío,
sin temor
que ni enemigo navío,
ni tormenta, ni bonanza
tu rumbo a torcer alcanza,
ni a sujetar tu valor.
Veinte presas
hemos hecho
a despecho
del inglés
y han rendido
sus pendones
cien naciones
a mis pies.
Que es mi barco mi tesoro,
que es mi Dios la libertad;
mi ley, la fuerza y el viento;
mi única patria, la mar.
Allá muevan feroz guerra
ciegos reyes
por un palmo más de tierra,
que yo tengo aquí por mío
cuanto abarca el mar bravío
a quien nadie impuso leyes.
Y no hay playa
sea cualquiera,
ni bandera
de esplendor,
que no sienta
mi derecho
y dé pecho
a mi valor
Que es mi barco mi tesoro,
que es mi Dios la libertad;
mi ley, la fuerza y el viento;
mi única patria, la mar.
A la voz de ¡barco viene!,
es de ver
cómo vira y se previene
a todo trapo a escapar:
que yo soy el rey del mar
y mi furia es de temer.
En las presas
yo divido
lo cogido
por igual:
sólo quiero
por riqueza
la belleza
sin rival.
Que es mi barco mi tesoro,
que es mi Dios la libertad;
mi ley, la fuerza y el viento;
mi única patria, la mar.
¡Sentenciado estoy a muerte!
Yo me río:
no me abandone la suerte,
y al mismo que me condena
colgaré de alguna antena
quizá en su propio navío.
Y si caigo,
¿qué es la vida?
Por perdida
ya la di
cuando el yugo
del esclavo
como un bravo sacudí.
Que es mi barco mi tesoro,
que es mi Dios la libertad;
mi ley, la fuerza y el viento;
mi única patria, la mar.
Son mi música mejor
aquilones,
el estrépito y temblor
de los cables sacudidos
del negro mar los bramidos
y el rugir de mis cañones.
Y del trueno
al son violento,
y del viento,
al rebramar,
yo me duermo
sosegado,
arrullado
por el mar.
Que es mi barco mi tesoro,
que es mi Dios la libertad;
mi ley, la fuerza y el viento;
mi única patria, la mar."""

In [2]:
def train_test_split(X, y):
    X_train = []
    X_test = []
    y_train = []
    y_test = []
    for i, elem in enumerate(X):
        if (i % 4) == 0:
            X_test.apend(elem)
            y_test.append(y(i))
        else:
            X_train.apend(elem)
            y_train.append(y(i))
    return X_train, X_test, y_train, y_test

In [9]:
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import keras.utils as ku 
import numpy as np
from sklearn.base import BaseEstimator

In [4]:
# source https://stackoverflow.com/questions/44697318/how-to-implement-perplexity-in-keras
def perplexity_raw(y_true, y_pred):
    """
    The perplexity metric. Why isn't this part of Keras yet?!
    https://stackoverflow.com/questions/41881308/how-to-calculate-perplexity-of-rnn-in-tensorflow
    https://github.com/keras-team/keras/issues/8267
    """
    cross_entropy = K.cast(
        K.equal(K.max(y_true, axis=-1), K.cast(K.argmax(y_pred, axis=-1), K.floatx())),
        K.floatx(),
    )
    perplexity = K.exp(cross_entropy)
    return perplexity


In [5]:
class BaseNetwork(BaseEstimator):
    def __init__(self, tokenizer=Tokenizer()):

        self.tokenizer = tokenizer

    def etl(self, data):

        # basic cleanup
        corpus = data.lower().split("\n")

        # tokenization
        self.tokenizer.fit_on_texts(corpus)
        self.total_words = len(self.tokenizer.word_index) + 1

        # create input sequences using list of tokens
        input_sequences = []
        for line in corpus:
            # TODO: Probar con fastText y HashingVectorizer y los demás de text de keras.
            # Onehot no hace falta si usamos embedding
            token_list = self.tokenizer.texts_to_sequences([line])[0]
            for i in range(1, len(token_list)):
                input_sequences.append(token_list[: i + 1])

        # pad sequences
        self.max_sequence_len = max([len(x) for x in input_sequences])
        input_sequences = np.array(
            pad_sequences(
                input_sequences, maxlen=self.max_sequence_len, padding="pre", value=0
            )
        )
        # create X and y
        X, y = input_sequences[:, :-1], input_sequences[:, -1]
        y = ku.to_categorical(y, num_classes=self.total_words)
        print(f"Shape of X: {X.shape}")
        print(f"Shape of y: {y.shape}")
        return X, y
    
    def generate_text(self, seed_text, next_words):

        for _ in range(next_words):
            token_list = self.tokenizer.texts_to_sequences([seed_text])[0]
            token_list = pad_sequences(
                [token_list], maxlen=self.max_sequence_len - 1, padding="pre"
            )
            predicted = self.net.predict_classes(token_list, verbose=0)

            output_word = ""
            for word, index in self.tokenizer.word_index.items():
                if index == predicted:
                    output_word = word
                    break
            seed_text += " " + output_word
        return seed_text


In [32]:
class Baseline(BaseNetwork):
    """Simple network with an embedding layer and a dense one"""
    def fit(
        self,
        X,
        y,
        earlystop=False,
        epochs=200,
        batch_size=None,
        verbose=1,
        activation="softmax",
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=[perplexity_raw],
    ):

        self.net = Sequential()
        self.net.add(Embedding(self.total_words, 64, input_length=X.shape[1]))
        self.net.add(Flatten())
        self.net.add(Dense(self.total_words, activation=activation))

        self.net.compile(loss=loss, optimizer=optimizer, metrics=metrics)
        print(self.net.summary())
        if earlystop:
            earlystop = EarlyStopping(
                monitor="val_loss", min_delta=0, patience=5, verbose=0, mode="auto"
            )
            self.net.fit(
                X,
                y,
                epochs=epochs,
                batch_size=None,
                verbose=verbose,
                callbacks=[earlystop],
            )
        else:
            self.net.fit(X, y, epochs=epochs, batch_size=None, verbose=verbose)

        return self


In [34]:
class LSTM_Embedding(BaseNetwork):
    """ LSTM Network """
    def fit(
        self,
        X,
        y,
        earlystop=False,
        epochs=200,
        batch_size=None,
        verbose=1,
        activation="softmax",
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=[perplexity_raw],
        hidden_lstm=1,
    ):
    
        self.net = Sequential()
        self.net.add(Embedding(self.total_words, 64))
        for _ in range(hidden_lstm):
            self.net.add(LSTM(32, input_shape=(self.max_sequence_len,), return_sequences=True))
        self.net.add(LSTM(32, input_shape=(self.max_sequence_len,)))
        self.net.add(Dense(self.total_words, activation=activation))
        self.net.compile(loss=loss, optimizer=optimizer, metrics=metrics)
        print(self.net.summary())
        if earlystop:
            earlystop = EarlyStopping(
                monitor="val_loss", min_delta=0, patience=5, verbose=0, mode="auto"
            )
            self.net.fit(
                X,
                y,
                epochs=epochs,
                batch_size=None,
                verbose=verbose,
                callbacks=[earlystop],
            )
        else:
            self.net.fit(X, y, epochs=epochs, batch_size=None, verbose=verbose)

        return self


In [38]:
model = Baseline()
X, y = model.etl(data)
model.fit(X, y, epochs=200)

Shape of X: (329, 6)
Shape of y: (329, 194)
Epoch 1/200
329/329 [==============================] - 0s 732us/step - loss: 5.2431 - perplexity_raw: 1.0940
Epoch 2/200
329/329 [==============================] - 0s 240us/step - loss: 5.1573 - perplexity_raw: 1.3029
Epoch 3/200
329/329 [==============================] - 0s 176us/step - loss: 5.0633 - perplexity_raw: 1.3081
Epoch 4/200
329/329 [==============================] - 0s 215us/step - loss: 4.9440 - perplexity_raw: 1.3656
Epoch 5/200
329/329 [==============================] - 0s 248us/step - loss: 4.7992 - perplexity_raw: 1.3917
Epoch 6/200
329/329 [==============================] - 0s 186us/step - loss: 4.6368 - perplexity_raw: 1.4648
Epoch 7/200
329/329 [==============================] - 0s 211us/step - loss: 4.4662 - perplexity_raw: 1.4439
Epoch 8/200
329/329 [==============================] - 0s 212us/step - loss: 4.3178 - perplexity_raw: 1.5797
Epoch 9/200
329/329 [==============================] - 0s 193us/step - loss: 4.1752 

Baseline(tokenizer=<keras.preprocessing.text.Tokenizer object at 0x7f5d81fb8f98>)

In [37]:
model = LSTM_Embedding()
X, y = model.etl(data)
model.fit(X, y, epochs=200, hidden_lstm=3)

Shape of X: (329, 6)
Shape of y: (329, 194)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, None, 64)          12416     
_________________________________________________________________
lstm_44 (LSTM)               (None, None, 32)          12416     
_________________________________________________________________
lstm_45 (LSTM)               (None, None, 32)          8320      
_________________________________________________________________
lstm_46 (LSTM)               (None, None, 32)          8320      
_________________________________________________________________
lstm_47 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_9 (Dense)              (None, 194)               6402      
Total params: 56,194
Trainable params: 56,194
Non-trainable params: 0
____________________________

LSTM_Embedding(tokenizer=<keras.preprocessing.text.Tokenizer object at 0x7f7d368b02e8>)

In [4]:
from ..src.model import BaseNetwork

ValueError: attempted relative import beyond top-level package

In [1]:
print("HOLA")

HOLA
